`#111`

<img width="950" height="200" alt="Image" src="https://github.com/user-attachments/assets/67ad5387-c218-4a22-b0ab-22653ef01bdf" />

https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

데이터셋은 [여기](https://github.com/user-attachments/files/20791799/tmdb_5000_movies.csv)에서 다운로드하세요 :)

위 TMDB 영화 데이터를 활용하여 다음 요구사항을 차례로 구현하고, overview를 통해 genre 멀티라벨을 예측하는 모델을 완성하세요. 🥇 

1. 데이터셋을 불러오고, 장르 정보를 리스트로 변환하는 코드를 작성하시오.

2. 줄거리(overview) 결측치와 장르 없는 영화를 제거하는 코드를 작성하시오.

3. 장르 정보를 다중 레이블 이진화하고, 줄거리 텍스트를 TF-IDF 벡터로 변환하는 코드를 작성하시오.

4. 훈련/테스트 데이터로 분할 후, 다중 레이블 분류 모델을 학습하는 코드를 작성하시오.

5. 테스트 데이터에 대해 예측을 수행하고, 평가 결과(classification_report)를 출력하시오.

6. 아래의 줄거리를 입력하여 예측된 장르를 출력하시오.
`"In the future, a robot assassin is sent back in time to kill a woman."`


In [ ]:
# 라이브러리 임포트 

import pandas as pd
import ast

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
# 데이터 로드
df = pd.read_csv("data/tmdb_5000_movies.csv")

# genres 컬럼: 문자열 → 리스트(dict) → 장르 이름 리스트
def parse_genres(genres_str):
    try:
        genres = ast.literal_eval(genres_str)
        return [g['name'] for g in genres]
    except:
        return []

df['genres_list'] = df['genres'].apply(parse_genres)

df[['title', 'genres_list']].head()


In [ ]:
# overview 결측 제거
df = df.dropna(subset=['overview'])

# 장르 없는 영화 제거
df = df[df['genres_list'].map(len) > 0]

df.shape


In [ ]:
# 장르 멀티라벨 이진화
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres_list'])

# 줄거리 TF-IDF 벡터화
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)

X = tfidf.fit_transform(df['overview'])

print("장르 개수:", len(mlb.classes_))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# 다중 레이블 분류 모델
model = OneVsRestClassifier(
    LogisticRegression(max_iter=1000)
)

model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

print(classification_report(
    y_test,
    y_pred,
    target_names=mlb.classes_
))


In [ ]:
new_overview = "In the future, a robot assassin is sent back in time to kill a woman."

# TF-IDF 변환
new_X = tfidf.transform([new_overview])

# 예측
pred = model.predict(new_X)

# 예측된 장르 출력
predicted_genres = mlb.inverse_transform(pred)

print("입력 줄거리:")
print(new_overview)
print("\n예측된 장르:")
print(predicted_genres[0])
